# AudioDataModule

In [1]:
import importlib
importlib.reload(importlib.import_module("src.dataModule"))
from src.dataModule import AudioDataModule
from transformers import AutoTokenizer
from transformers import WhisperTokenizer
from prep.processor import get_tokenizer

dm = AudioDataModule()
dm.setup("fit")
dm.setup("validate")
dm.setup("test")

# Batch

In [2]:
from tqdm import tqdm
import time

start_time = time.time()
for batch in tqdm(dm.train_dataloader()):
    x, y = batch
    pass

print("--- %s seconds ---" % (time.time() - start_time))

100%|██████████| 1/1 [00:00<00:00,  6.94it/s]

--- 0.15218591690063477 seconds ---


---
# Test

In [8]:
from datasets import load_dataset
dataset = load_dataset("google/fleurs", "he_il", split="test", cache_dir='datasets/test', trust_remote_code=True)
dataset = dataset.rename_column("transcription", "normalized_text")

In [2]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-100h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-100h")

Some weights of the model checkpoint at facebook/wav2vec2-base-100h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.mask_time_emb_vector']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-100h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav

In [3]:
audio_sample = dataset[2]
inputs = processor(audio_sample["audio"]["array"], sampling_rate=audio_sample["audio"]["sampling_rate"], return_tensors="pt")

In [4]:
x = inputs.input_values

with torch.no_grad():
  logits = model(x).logits

  

In [1]:
from prep.processor import get_tokenizer
from charactertokenizer import CharacterTokenizer
from string import whitespace, punctuation
from hebrew.chars import HEBREW_CHARS
from transformers import WhisperFeatureExtractor
# tokenizer = get_tokenizer()


tokenizer = get_tokenizer()

example = "!ההפירמידהה                         יום"

tokens = tokenizer(example)["input_ids"]
print(tokens)

print(tokenizer.decode(tokens, skip_special_tokens=True))

[0, 6, 11, 11, 26, 16, 31, 20, 16, 10, 11, 11, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 16, 12, 21, 1]
ההפירמידהה                         יום


In [9]:
import torch

def decode(encoding):
    """
    normal decode function
    """
    text = tokenizer.decode(encoding, skip_special_tokens=True)
    return text

In [12]:
print(decode(0))

[CLS]


--
# LLM

In [34]:
from transformers import BertModel, BertTokenizerFast, pipeline

tokenizer = BertTokenizerFast.from_pretrained('onlplab/alephbert-base')
model = BertModel.from_pretrained('onlplab/alephbert-base')

sentiment_analysis = pipeline(
    task="token-classification",
    model="onlplab/alephbert-base",
    tokenizer="onlplab/alephbert-base",
)

print("\n\n")
print(sentiment_analysis('אני מתלבט מה לאכול לארוחת צהריים'))

Some weights of BertModel were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of BertForTokenClassification were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.





[{'entity': 'LABEL_1', 'score': 0.71913487, 'index': 1, 'word': 'אני', 'start': 0, 'end': 3}, {'entity': 'LABEL_0', 'score': 0.520349, 'index': 2, 'word': 'מתלבט', 'start': 4, 'end': 9}, {'entity': 'LABEL_1', 'score': 0.56510997, 'index': 3, 'word': 'מה', 'start': 10, 'end': 12}, {'entity': 'LABEL_1', 'score': 0.73355114, 'index': 4, 'word': 'לאכול', 'start': 13, 'end': 18}, {'entity': 'LABEL_1', 'score': 0.6801301, 'index': 5, 'word': 'לארוחת', 'start': 19, 'end': 25}, {'entity': 'LABEL_1', 'score': 0.7803837, 'index': 6, 'word': 'צהריים', 'start': 26, 'end': 32}]


In [33]:
from torchaudio.models.decoder import download_pretrained_files
import torch

files = download_pretrained_files("librispeech-4-gram")

print(files)

model = torch.load('/home/zeus/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/lm.bin')

PretrainedFiles(lexicon='/home/zeus/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/lexicon.txt', tokens='/home/zeus/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/tokens.txt', lm='/home/zeus/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/lm.bin')


/tmp/ipykernel_716738/4032573503.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/home/zeus/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-

UnpicklingError: invalid load key, 'm'.